In [64]:
import math
import random

In [65]:
def sum(data: list[float]) -> float:
    result = 0
    for item in data:
        result += item
    return result

In [66]:
def average(data: list[float]) -> float:
    return (1 / len(data)) * sum(data)

In [67]:
def standard_deviation(data: list[float]) -> float:
    deviations = []
    mean = average(data)
    for item in data:
        deviations.append(math.pow(item - mean, 2))
    deviations_sum = sum(deviations)
    return math.sqrt(deviations_sum / (len(data) - 1))

In [68]:
def standard_deviation_from_the_mean(data: list[float]) -> float:
    return standard_deviation(data) / math.sqrt(len(data))

In [74]:
from ctypes import ArgumentError

def mmq_sigma_square(y_errors: list[float]) -> float:
    items_to_sum = [1 / i ** 2 for i in y_errors]
    return sum(items_to_sum)

def mmq_x(xs: list[float], y_errors: list[float]) -> float:
    if (len(xs) != len(y_errors)):
        raise ArgumentError(xs)
    items_to_sum = []
    for xi, y_error_i in zip(xs, y_errors):
        items_to_sum.append(xi / y_error_i ** 2)
    return (1 / mmq_sigma_square(y_errors)) * sum(items_to_sum)

def mmq_x_square(xs: list[float], y_errors: list[float]) -> float:
    if (len(xs) != len(y_errors)):
        raise ArgumentError(xs)
    items_to_sum = []
    for xi, y_error_i in zip(xs, y_errors):
        items_to_sum.append(xi ** 2 / y_error_i ** 2)
    return (1 / mmq_sigma_square(y_errors)) * sum(items_to_sum)

def mmq_y(ys: list[float], y_errors: list[float]) -> float:
    if (len(ys) != len(y_errors)):
        raise ArgumentError(ys, y_errors)
    items_to_sum = []
    for yi, y_error_i in zip(ys, y_errors):
        items_to_sum.append(yi / y_error_i ** 2)
    return (1 / mmq_sigma_square(y_errors)) * sum(items_to_sum) 

def mmq_xy(xs: list[float], ys: list[float], y_errors: list[float]) -> float:
    if (not (len(xs) == len(ys) and len(ys) == len(y_errors))):
        raise ArgumentError(ys, xs, y_errors)
    items_to_sum = []
    for xi, yi, y_error_i in zip(xs, ys, y_errors):
        items_to_sum.append(xi * yi / y_error_i ** 2)
    return (1 / mmq_sigma_square(y_errors)) * sum(items_to_sum)

def mmq_a(xs: list[float], ys: list[float], y_errors: list[float]) -> float:
    x = mmq_x(xs, y_errors)
    y = mmq_y(ys, y_errors)
    xy = mmq_xy(xs, ys, y_errors)
    x_square = mmq_x_square(xs, y_errors)
    return (x * y - xy) / (x ** 2 - x_square)

def mmq_b(xs: list[float], ys: list[float], y_errors: list[float]) -> float:
    return mmq_y(ys, y_errors) - mmq_a(xs, ys, y_errors) * mmq_x(xs, y_errors)

def mmq_delta_a(xs: list[float], y_errors: list[float]) -> float:
    sigma_square = mmq_sigma_square(y_errors)
    x = mmq_x(xs, y_errors)
    x_square = mmq_x_square(xs, y_errors)
    return math.sqrt((1 / sigma_square) / (x_square - x ** 2))

def mmq_delta_b(xs: list[float], y_errors: list[float]) -> float:
    sigma_square = mmq_sigma_square(y_errors)
    x = mmq_x(xs, y_errors)
    x_square = mmq_x_square(xs, y_errors)
    return math.sqrt((x_square / sigma_square) / (x_square - x ** 2))

ys = [14.0, 32.0, 50, 68.0]
y_errors = [1.0, 1.4, 1.7, 2]
xs = [0.5, 0.7, 0.8, 0.8]
print(f"Sigma square: {mmq_sigma_square(y_errors)}")
print(f"X: {mmq_x(xs, y_errors)}")
print(f"X square: {mmq_x_square(xs, y_errors)}")
print(f"Y: {mmq_y(ys, y_errors)}")
print(f"XY: {mmq_xy(xs, ys, y_errors)}")
print(f"a: {mmq_a(xs, ys, y_errors)}")
print(f"b: {mmq_b(xs, ys, y_errors)}")
print(f"deltaA: {mmq_delta_a(xs, y_errors)}")
print(f"deltaB: {mmq_delta_b(xs, y_errors)}")

Sigma square: 2.106224842878328
X: 0.6333414358157663
X square: 0.41849914085746615
Y: 30.684078622019197
XY: 21.778017685763377
a: 134.91487929210305
b: -54.763104741752144
deltaA: 5.226977797374939
deltaB: 3.381410841254819


In [73]:
## Relatório 2 - FEMEC

def erro_dos_intervalos(comprimentos: list[float]) -> float:
    return math.sqrt(standard_deviation_from_the_mean(comprimentos) ** 2 + 0.05 ** 2)

def erro_cronometro(tempos: list[float]) -> float:
    return math.sqrt(standard_deviation_from_the_mean(tempos) ** 2 + 0.01 ** 2)

def erro_velocidade(l: float, erroL: float, t: float, erroT: float) -> float:
    return (l / t) * math.sqrt((erroL / l) ** 2 + (erroT / t) ** 2)
    
def show(intervalo: str, comprimentos: list[float], tempos: list[float]) -> None:
    print(intervalo)
    mediaComprimentos = average(comprimentos)
    incertezaComprimentos = erro_dos_intervalos(comprimentos)
    mediaTempos = average(tempos)
    incertezaTempos = erro_cronometro(tempos)
    v = mediaComprimentos / mediaTempos
    erroV = erro_velocidade(mediaComprimentos, incertezaComprimentos, mediaTempos, incertezaTempos)
    print(f"Média de comprimento = {round(mediaComprimentos, 2)} +- {round(incertezaComprimentos, 2)}")
    print(f"Média de tempo = {round(mediaTempos, 2)} +- {round(incertezaTempos, 2)}")
    print(f"Velocidade Média: {round(v, 2)} +- {round(erroV, 2)}")
    print()
    
l1_medidas = [15.3, 13.7, 11.5]
t1_medidas = [0.735, 0.727, 0.708]
l2_medidas = [20.0, 18.8, 16.2]
t2_medidas = [0.398, 0.398, 0.398]
l3_medidas = [19.5, 17.9, 16.21]
t3_medidas = [0.300, 0.299, 0.299]
l4_medidas = [19.4, 17.8, 16.0]
t4_medidas = [0.243, 0.244, 0.243]

show("L1", l1_medidas, t1_medidas)
show("L2", l2_medidas, t2_medidas)
show("L3", l3_medidas, t3_medidas)
show("L4", l4_medidas, t4_medidas)

L1
Média de comprimento = 13.5 +- 1.1
Média de tempo = 0.72 +- 0.01
Velocidade Média: 18.66 +- 1.56

L2
Média de comprimento = 18.33 +- 1.12
Média de tempo = 0.4 +- 0.01
Velocidade Média: 46.06 +- 3.05

L3
Média de comprimento = 17.87 +- 0.95
Média de tempo = 0.3 +- 0.01
Velocidade Média: 59.7 +- 3.75

L4
Média de comprimento = 17.73 +- 0.98
Média de tempo = 0.24 +- 0.01
Velocidade Média: 72.88 +- 5.03

